In [1]:
import pandas as pd
import os
import json


In [8]:

def get_video_data(host_folder) -> pd.DataFrame:
    # List files in output/{asset_id} dir
    df_list = []
    files = os.listdir(host_folder)
    for file in files:
        if file.endswith('.json'):
            print(file)
            path = f'{host_folder}/{file}'
            df_data = pd.read_json(path)
            df_list.append(df_data)

    df = pd.concat(df_list)
    
    return df
    
host_folder = '../data/inputs'



df = get_video_data( host_folder =  host_folder)
print(df.shape)

df.head()

tmp8_r9nace_chunk_2_of_4350.mp4_results.json
tmp8_r9nace_chunk_3_of_4350.mp4_results.json
tmp8_r9nace_chunk_1_of_4350.mp4_results.json
tmp8_r9nace_chunk_0_of_4350.mp4_results.json
tmp8_r9nace_chunk_4_of_4350.mp4_results.json
(21750, 4)


,annotated_video,total_frames,data,fps
0,tmp8_r9nace_chunk_2_of_4350_annotated.mp4,4350,"{'results': [{'name': 'car', 'class': 2, 'conf...",29
1,tmp8_r9nace_chunk_2_of_4350_annotated.mp4,4350,"{'results': [{'name': 'car', 'class': 2, 'conf...",29
2,tmp8_r9nace_chunk_2_of_4350_annotated.mp4,4350,"{'results': [{'name': 'car', 'class': 2, 'conf...",29
3,tmp8_r9nace_chunk_2_of_4350_annotated.mp4,4350,"{'results': [{'name': 'car', 'class': 2, 'conf...",29
4,tmp8_r9nace_chunk_2_of_4350_annotated.mp4,4350,"{'results': [{'name': 'car', 'class': 2, 'conf...",29


In [ ]:
df['data'][0]

In [ ]:
inner_data = []


for index, row in df.iterrows():
    # Load the JSON data from the 'data' column
    data = row['data']
    
    print(data)
    print(data.keys())
    
    for result in data['results']:
        result['frame_number'] = data['frame_number']
        result['original_frame'] = data['original_frame']
        result['s3_path'] = data['s3_path']
        result['total_frames'] = data['total_frames']
        result['annotated_video'] = data['annotated_video']
        
        inner_data.append(result)
    
    print(inner_data)


    

In [10]:
def clean_data(df: pd.DataFrame) -> pd.DataFrame:
    # Create an empty list to store the parsed data
    inner_data = []

    for index, row in df.iterrows():
        # Load the JSON data from the 'data' column
        data = row['data']
        
        for result in data['results']:
            result['frame_number'] = data['frame_number']
            result['original_frame'] = data['original_frame']
            result['s3_path'] = data['s3_path']
            result['fps'] = row['fps']
            
            result['total_frames'] = row['total_frames']
            result['annotated_video'] = row['annotated_video']
            
            inner_data.append(result)

    # Create a DataFrame from the flattened list of dictionaries
    data_df = pd.json_normalize(inner_data)

    return data_df

df_c = clean_data(df)
print(df_c.shape)
df_c.head()

(135300, 14)


,name,class,confidence,track_id,frame_number,original_frame,s3_path,fps,total_frames,annotated_video,box.x1,box.y1,box.x2,box.y2
0,car,2,0.84253,1,0,tmp8_r9nace_chunk_2_of_4350_annotated_images/t...,5049e5f6-ec91-4afb-b2f5-a63a991a7993/images/tm...,29,4350,tmp8_r9nace_chunk_2_of_4350_annotated.mp4,334.07565,48.26264,405.30865,87.45946
1,car,2,0.55709,3,0,tmp8_r9nace_chunk_2_of_4350_annotated_images/t...,5049e5f6-ec91-4afb-b2f5-a63a991a7993/images/tm...,29,4350,tmp8_r9nace_chunk_2_of_4350_annotated.mp4,375.12015,38.97644,418.67746,64.04517
2,car,2,0.66877,4,0,tmp8_r9nace_chunk_2_of_4350_annotated_images/t...,5049e5f6-ec91-4afb-b2f5-a63a991a7993/images/tm...,29,4350,tmp8_r9nace_chunk_2_of_4350_annotated.mp4,411.59988,31.60214,434.65320,51.49309
3,bus,5,0.13255,56,0,tmp8_r9nace_chunk_2_of_4350_annotated_images/t...,5049e5f6-ec91-4afb-b2f5-a63a991a7993/images/tm...,29,4350,tmp8_r9nace_chunk_2_of_4350_annotated.mp4,68.63795,67.93243,316.96194,224.10532
4,person,0,0.38228,50,0,tmp8_r9nace_chunk_2_of_4350_annotated_images/t...,5049e5f6-ec91-4afb-b2f5-a63a991a7993/images/tm...,29,4350,tmp8_r9nace_chunk_2_of_4350_annotated.mp4,259.65268,115.14301,310.29587,199.90434


In [6]:
df_join = df.join(df_c)
# Remove "data" column from the DataFrame
df_join = df_join.drop(columns=['data'])
print(df_join.shape)
df_join.head()

(21750, 14)


,annotated_video,total_frames,fps,name,class,confidence,track_id,frame_number,original_frame,s3_path,box.x1,box.y1,box.x2,box.y2
0,tmp8_r9nace_chunk_2_of_4350_annotated.mp4,4350,29,car,2,0.84253,1,0,tmp8_r9nace_chunk_2_of_4350_annotated_images/t...,5049e5f6-ec91-4afb-b2f5-a63a991a7993/images/tm...,334.07565,48.26264,405.30865,87.45946
1,tmp8_r9nace_chunk_2_of_4350_annotated.mp4,4350,29,car,2,0.55709,3,0,tmp8_r9nace_chunk_2_of_4350_annotated_images/t...,5049e5f6-ec91-4afb-b2f5-a63a991a7993/images/tm...,375.12015,38.97644,418.67746,64.04517
2,tmp8_r9nace_chunk_2_of_4350_annotated.mp4,4350,29,car,2,0.66877,4,0,tmp8_r9nace_chunk_2_of_4350_annotated_images/t...,5049e5f6-ec91-4afb-b2f5-a63a991a7993/images/tm...,411.59988,31.60214,434.65320,51.49309
3,tmp8_r9nace_chunk_2_of_4350_annotated.mp4,4350,29,bus,5,0.13255,56,0,tmp8_r9nace_chunk_2_of_4350_annotated_images/t...,5049e5f6-ec91-4afb-b2f5-a63a991a7993/images/tm...,68.63795,67.93243,316.96194,224.10532
4,tmp8_r9nace_chunk_2_of_4350_annotated.mp4,4350,29,person,0,0.38228,50,0,tmp8_r9nace_chunk_2_of_4350_annotated_images/t...,5049e5f6-ec91-4afb-b2f5-a63a991a7993/images/tm...,259.65268,115.14301,310.29587,199.90434
